# Data Processing and model building

일단 빠른 진행을 위해 reweight 을 적용하지 않기로 한 상태.(tfidf 관련 feature 들은 모두 사용되지 않게 될 것 같음.)

이렇게 진행하기 위해선 기존의 코드를 전면적으로 수정해야하기 때문에 필요한 코드들을 여기에 복붙하고 수정해서 사용하는 방식으로 진행함.

수정된 부분은 최대한 그 목차에서 설명하겠음



> 주의 : 수정하고 싶으면 반드시 사본을 만들어서 거기에 수정하기



## Fake News Detection 코드

여기서 우리 프로세스는

1. 전처리 및 Co-Occurrence 데이터 프레임 생성
- 여기서 전처리를 진행할때 입력값은 TiCNN 데이터 셋의 각 본문이 될 것임.
- 각 기사에 대한 Co-Occurrence 데이터 프레임이 그대로 메모리에 남아있으면 2만개나 되니 메모리 터질 우려가 있어 일단 우리 공유 드라이브에 각각을 저장해둘거임 (아래 코드 참조)


2. 만들어진 데이터프레임을 불러와서 각각 원래 만들어둔 피쳐 만들고 Ti CNN 데이터 프레임에 추가해주기 

- 내가 진행못함

### nltk download(시작하기 전에 실행)

In [1]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd 'drive/Shared drives/2019_URP_2/data'
!ls

/content/drive/Shared drives/2019_URP_2/data
all_data_mini.csv      cooc	  cooc.zip   re_cooc
all_data_modified.csv  cooc_mini  glove.42B  tfidf


### Cooccurrence.py (수정사항 확인)(새로 돌릴 시 진행)

원본에서 수정사항

1. tag filter 없이 tag 의 첫글자가 (startswith 함수 이용) V, N, R, J 인 경우만 선택하는 것으로 바꿈(select result 함수)

2. tag filter 변수 init에서 전부 삭제

3. sentent.replace 에 Donald J. Trump 추가





In [0]:
from collections import Counter, defaultdict
from itertools import combinations
import nltk

import ast
import pandas as pd
from nltk import sent_tokenize, word_tokenize, pos_tag, ne_chunk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tree import Tree

class Processing():
    def __init__(self):
        self.tag_map = defaultdict(lambda: wn.NOUN)
        self.tag_map['J'] = wn.ADJ
        self.tag_map['V'] = wn.VERB
        self.tag_map['R'] = wn.ADV

    @staticmethod
    def apply_collocations(sentence):
        sentence = sentence.replace("George H.W. Bush", "George_H.W._Bush")
        sentence = sentence.replace("Donald J. Trump", "Donald_J._Trump")
        # add any phrase here
        return sentence

    # 문장 하나 lemmatization 함수
    def lemma_sentence(self, text):  # token에 is, 같은 애들을 be 로 변환 시키지 않음
        results = []
        tokens = word_tokenize(text)
        # NER_chunk 함수 넣어주기 (tokens 단위?)
        tokens = self.ner_chunk(tokens)  #-> pos tag / ne_chunk 포함
        lmtzr = WordNetLemmatizer()
        replace_data={"n't":'not'} #lemmatatization에서 제거 되고 싶지 않은 단어 추가
        for token, tag in pos_tag(tokens):

            # print("token :", token, "tag :", tag)
            if token in replace_data.keys():
                # print("pass replace_Data: ",token)
                token =token.replace(token,replace_data[token])
                # print("after replace: ",token)
            lemma = lmtzr.lemmatize(token, self.tag_map[tag[0]])
            # print(token, "=>", lemma)
            results.append(lemma)
        return results

    # 문서 전체 lemmatization 함수
    def lemma_text(self, text):
        # collocation 을 이 단에서 추가해야할 듯 (sent tokenize 되지 않도록)
        lemma_data = []
        sentences = sent_tokenize(text)
        for sent in sentences:
            lemma_sent = self.lemma_sentence(sent)
            lemma_data.append(lemma_sent)
        return lemma_data

    # 불용어 처리 함수
    # 여기서부턴 string형태가 아니라 이중리스트 형태이므로 sentences 와 sentence 로 구분함
    def stopword(self, sentences):
        stopWords = set(stopwords.words('english'))-set(['not'])
        added_stopword = ['“', '”', '.', ',', '-', "—", "–", "'s", "n't", "''", ';', '&', "``", '?', "‘", "’"]
        results = []

        for sentence in sentences:
            wordsFiltered = []
            wordsStopped = []
            for w in sentence:
                if w not in stopWords and w not in added_stopword and not w.isdigit():
                    wordsFiltered.append(w)
                else:
                    wordsStopped.append(w)
            results.append(wordsFiltered)

        # print(results)
        return results

    # 태깅 함수

    # apply_collocation 수정
    def ner_chunk(self,tokens): #George H.W. Bush는 따로 작업
        chunked = ne_chunk(pos_tag(tokens), binary=True)
        # prev = None
        continuous_chunk = []
        current_chunk = []
        # print("gcc_text in :", text)
        # print("gcc chunked in : ",chunked )
        for i in chunked:
            if type(i) == Tree:
                current_chunk.append("_".join([token for token, pos in i.leaves()]))
                # print("current chunk: ",current_chunk,"\n")
                named_entity = " ".join(current_chunk)
                continuous_chunk.append(named_entity)
                current_chunk = []
            else:
                continuous_chunk.append(i[0])
                continue
        if current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        return continuous_chunk

    def tag_content(self, sentences):
        """
        Tag all words in content
        :param sentences:(list) processed data
        :return: (list) tagged words divided by each sentence
        """
        results = []
        for sentence in sentences:
            tagged_content = pos_tag(sentence)
            results.append(tagged_content)

        return results

    # 태그 결과에서 필터링하는 함수
    def select_results(self, sentences):
        """
        Select word by filtering certain tags
        :param sentences: (list) processed data
        :param tag_filter: (list) tags which should be left
        :return: (list) words divided by each sentence
        """

        selected_results = []

        for sentence in sentences:
            selection = []
            # 단어를 lex, tag 를 cat 이라 표현
            for lex, tag in sentence:
                if tag.startswith('V') or tag.startswith('N') or tag.startswith('R') or tag.startswith('J') :
                    # tag 말고 안에 단어 lex 만 남겨야함
                    selection.append(lex)

            if len(selection) > 0:
                selected_results.append(selection)
        return selected_results

    # Co-occurence matrix 생성 함수
    def create_cooc_mat(self, sentences):
        """
        Create Co-Occurrence Matrix
        :param sentences: (list of list) processed data
        :return: (list) The number of times two words occur together in each sentence in a document. [(word1, word2), count]
        """
        word_cooc_mat = Counter()
        for sentence in sentences:
            for w1, w2 in combinations(sentence, 2):
                if len(w1) == 1 or len(w2) == 1:  # 1음절 단어 제외
                    continue
                if w1 == w2:  # 동일한 단어 벡터는 계산 x.
                    continue
                elif word_cooc_mat[(w2, w1)] >= 1:  # 동일한 조합이고 순서만 반대인 경우
                    word_cooc_mat[(w2, w1)] += 1  # 처음 했던 조합에 카운트하겠다
                    # print(word_cooc_mat[(w2, w1)])
                else:
                    word_cooc_mat[(w1, w2)] += 1
                    # print(word_cooc_mat[(w1, w2)])

        # list_key_value = [[k,v] for k, v in word_cooc_mat.items()]

        list_keys = [k for k in word_cooc_mat.keys()]
        list_values = [v for v in word_cooc_mat.values()]
        conv_list_keys = [[w1, w2] for w1, w2 in list_keys]
        linkages = pd.Series(two_words for two_words in conv_list_keys)
        weights = pd.Series(list_values)
        data = pd.DataFrame({'Linkage': linkages, 'Weight': weights})
        sorted_data = data.sort_values(by=['Weight'], ascending=False)
        # return list_key_value
        return sorted_data

    def cooc(self, filepath=None, text=None):
        if filepath is not None:
            text = open(filepath, encoding='utf-8').read()
        else:
            text = text

        text = self.apply_collocations(text)
        lem_sents = self.lemma_text(text)
        stop_sents = self.stopword(lem_sents)
        tag_sents = self.tag_content(stop_sents)
        sel_sents = self.select_results(tag_sents) # 단어 리스트 - 사용할 품사 종류 합의 필요
        cooc_mat = self.create_cooc_mat(sel_sents) # 단어간 연결 데이터프레임
        return sel_sents, cooc_mat

## Ti CNN 데이터 다루기


데이터를 구글 드라이브에서 불러올건데 미리 우리 공유 폴더에 share drive로 만들어둠. 

데이터를 딱 보면 텍스트에 <U+2019> 등등 이상한 단어들이 보임. 정규표현식을 사용해서 없어준뒤 우리가 원하는 것들 진행할 거임



### 전체 진행중 (완료)

100개 진행시 28초 이므로 20000개 진행시 약 5600초, 정도 걸릴 것이라 예상됨. 내일 아침에 와서 다시 진행할 수 있을 정도 수준이므로 일단 돌려놓기



** 이거까지 끝나면 우리 2019 urp / data 폴더 안에 19919 개의 csv 가 만들어져 있어야함.

In [0]:
import os
import time
from tqdm import tqdm_notebook    


N = Processing()
start = time.time()
os.system('mkdir cooc/')
for i in tqdm_notebook(df.index): # 18418개 진행 
    _, cooc = N.cooc(text=df.loc[i, 'text'])
    cooc.to_csv('cooc/' + str(i) + '.csv')
end = time.time()
print("It takes :", end - start)

It takes : 3163.0234892368317


In [0]:
x = os.listdir('cooc/')
print(len(x))

18418


### 연어 처리 

0909 한석 작업 - 수정 필요**굵은 텍스트**

Test용 완성된 cooc 불러오기

In [0]:
temp = pd.read_csv('cooc/1262.csv',index_col=0)
print("len temp: ",len(temp))
temp.head()

In [0]:
# String type 과 List type 일 시 차이 비교

print("String type: ",'Trump' in "['Donald_Trump', 'foreign']")
print("List type: ",'Trump' in ['Donald_Trump', 'foreign'])

String type:  True
List type:  False


In [0]:
# string to list 로 바꿔주는 module
# ast.literal_eval() 함수 사용

import ast

def strToList(df):
    '''
    string type으로 값을 가지고 있는 df를 list type으로 변경해줌
    '''
    try :
        df['Linkage']= [ast.literal_eval(str) for str in df.iloc[:,0]]
        
    except:
        print("Already Transformed to list")
        pass
    
    return df


# temp = strToList(temp)
# print(type(temp['Linkage'][0]))
# temp.head()

In [0]:
# 전체 문서 내 corpus에서 연어인 단어들을 추출
print("'_'가 들어가는 linkage의 단어들 추출")
print(set(sum([ast.literal_eval(s) for s in temp.iloc[:,0] if "_" in s],[])) )

colloc_words = [i for i in set(sum([ast.literal_eval(s) for s in temp.iloc[:,0] if "_" in s],[])) if '_' in i]
print("연어인 단어들만 추출")
print(colloc_words)

findindex 함수 적용 후 colloc 에 해당하는 단어를 못찾는 오류발생 -> 해결완료? 다시 체크

해결필요 - 적용 전 전체 index : 98, colloc index :17

In [0]:
# 연어별 index 찾기 함수화 필요
# 분절된 단어는 포함되며, 연어가 아닌 linkage 갖는 index 찾기

def findIndex(colloc_word,df):
    '''
    > params: 
    colloc_word - 연어 ex) 'United_States', 'Donald_Trump', 'Fox_News', 'Hillary_Clintons', 'Middle_East'
    df - strToList 처리가 되지 않은 dataFrame
    
    '''
    
    a, b = colloc_word.split('_')
    print("a= ",a,",b= ",b)
    print("")
#     df = strToList(df) # string to index 화
    

    colloc_count=0
    total_count=0
    colloc_index =[]
    total_index=[]
    no_colloc_index =[]
    
    print("df 길이: ",len(df))   
    for i in range(len(df)):
        if (a in df.iloc[i,0]) and (b in df.iloc[i,0]):
            colloc_count+=1
            colloc_index.append(df.index[i])

        if (a in df.iloc[i,0]) or (b in df.iloc[i,0]):
            total_count+=1
            total_index.append(df.index[i])
            
            
    no_colloc_index = [i for i in total_index if i not in colloc_index]
    
    
#     print("연어가 포함된 index 수: ",colloc_count)
#     print("전체 index 수 :", total_count)
#     print("연어가 있는 index :", colloc_index)
#     print("전체 index :", total_index)
#     print("연어가 없는 index: ",no_colloc_index)
   
    
    return colloc_index, no_colloc_index
    

#### a) findIndex 함수 적용버전

In [0]:
# string type일 때와 달라짐 - 수정 필요

colloc_index, no_colloc_index = findIndex('Donald_Trump',temp)
print("colloc_index", len(colloc_index),'개 ',colloc_index)
print("no_colloc_index",len(no_colloc_index),"개 ",no_colloc_index)

a=  Donald ,b=  Trump

df 길이:  1221
colloc_index 17 개  [63, 796, 797, 798, 799, 800, 801, 802, 803, 822, 823, 824, 833, 839, 842, 844, 845]
no_colloc_index 81 개  [2, 4, 5, 12, 13, 16, 17, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 49, 53, 57, 65, 71, 89, 101, 147, 154, 198, 470, 486, 487, 488, 489, 490, 500, 502, 503, 509, 516, 517, 527, 529, 530, 531, 532, 533, 536, 545, 553, 560, 567, 568, 728, 734, 735, 736, 737, 738, 739, 740, 741, 743, 744, 745, 884, 888, 930, 931, 932, 946, 951, 954, 998, 1042, 1060, 1067, 1069, 1070, 1071, 1072, 1073, 1074]


In [0]:
# string type일 때와 달라짐 - 수정 필요

colloc_index, no_colloc_index = findIndex('Donald_Trump',temp)
print("colloc_index", len(colloc_index),'개 ',colloc_index)
print("no_colloc_index",len(no_colloc_index),"개 ",no_colloc_index)

a=  Donald ,b=  Trump

df 길이:  1221
colloc_index 17 개  [33, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 18, 2, 50, 49, 48]
no_colloc_index 81 개  [330, 329, 587, 972, 162, 164, 154, 1108, 1109, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1099, 850, 892, 331, 338, 323, 834, 879, 904, 865, 1163, 1172, 1173, 1174, 1175, 1176, 915, 973, 971, 975, 974, 976, 977, 982, 981, 980, 979, 978, 952, 951, 933, 941, 949, 950, 303, 328, 337, 336, 335, 334, 333, 332, 327, 326, 325, 324, 163, 158, 339, 340, 341, 530, 545, 559, 514, 572, 593, 592, 591, 590, 589, 588, 586, 585]


In [0]:
# 연어가 있던 index - b)findIndex 미적용 시
# [63, 796, 797, 798, 799, 800, 801, 802, 803, 822, 823, 824, 833, 839, 842, 844, 845]

# 연어가 없던 index -b) findindex 미적용 시
#[2, 4, 5, 12, 13, 16, 17, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 49, 53, 57, 65, 71, 89, 101, 147, 154, 198, 470, 486, 487, 488, 489, 490, 500, 502, 503, 509, 516, 517, 527, 529, 530, 531, 532, 533, 536, 545, 553, 560, 567, 568, 728, 734, 735, 736, 737, 738, 739, 740, 741, 743, 744, 745, 884, 888, 930, 931, 932, 946, 951, 954, 998, 1042, 1060, 1067, 1069, 1070, 1071, 1072, 1073, 1074]

In [0]:
temp.iloc[no_colloc_index,0]

## MergeColloc class 형성

- findCollocWords
- strToList
- findindex 
- findLinkageWord

순서 : findCollocWords -> findindex -> strToList -> findLinkageWord -> SumDrop ->SumOthers

strToList 함수는 data를 합칠 때 사용하면 될 듯

In [0]:
import ast
class MergeColloc():
    def __init__(self,data_path):
        root_dir ='cooc/'
        self.df = pd.read_csv(root_dir + data_path, index_col=0)

        
        # Linkage : string type에서 형성
        temp_list = [i for i in set(sum([ast.literal_eval(s) for s in self.df.iloc[:,0] if "_" in s],[])) if '_' in i]
        # 두 개의 단어로만 형성된 연어만 취급
        self.colloc_words =[word for word in temp_list if len(word.split('_'))==2]
        
            
#         print("colloc_words in corpus: ",self.colloc_words)
        
    def findIndex(self,colloc_word,df=[]):
        '''
        * params: 
        colloc_word - Target collocation word. ex) 'United_States', 'Donald_Trump', 'Fox_News', 'Hillary_Clintons', 'Middle_East'
        df - A data frame which wasn't applied 'strToList' function.
 
        * return : 
        colloc_index - indexes which have collocation words
        no_colloc_index - indexes which don't have collocation words, but splited word of them.
        
        '''
        if len(df) ==0:
            df = self.df
            
        
        a, b = colloc_word.split('_')
#         print("a= ",a,",b= ",b)
   
        colloc_index =[]
        no_colloc_index=[]
        nothing_index = []
        total_index=[]

        # 추가수정 - row 값에서 index 값 반환으로 변경
        # string type 버젼
        for i in range(len(df)):
            if (colloc_word in df.iloc[i,0]):
                colloc_index.append(df.index[i])
                      
            if (a in df.iloc[i,0]) or (b in df.iloc[i,0]):
                total_index.append(df.index[i])
                
            if (a not in df.iloc[i:0]) and (b not in df.iloc[i,0]):
                nothing_index.append(df.index[i])
                

        no_colloc_index = [i for i in total_index if i not in colloc_index]
        
        return colloc_index, no_colloc_index , nothing_index
        
        
        ### 0915 수정
        # 수정 필요 - query 문? / append 대체? / list comprehension
        
    
    def findIndexListType(self,colloc_word,df=[]):
        if len(df) ==0:
            df =self.df
        a, b = colloc_word.split('_')
        df = strToList(df) # string to index 화

        colloc_index =[]
        no_colloc_index=[]
        nothing_index = []
        total_index=[]
        
        # list type 버젼
        for i in range(len(df)):
            if (colloc_word in df.iloc[i,0]):
                colloc_index.append(df.index[i])
                      
            if (a in df.iloc[i,0]) or (b in df.iloc[i,0]):
                total_index.append(df.index[i])
                
            if (a not in df.iloc[i:0]) and (b not in df.iloc[i,0]):
                nothing_index.append(df.index[i])
                

        no_colloc_index = [i for i in total_index if i not in colloc_index]
        
        return colloc_index, no_colloc_index , nothing_index
    
        ### 0915 수정
        # 수정 필요 
    
    


    def findLinkageWord(self,colloc_word,df=[]):
        '''
        params: colloc_word - A collocation word. Same word as findIndex function.
        return: duplicated_word - The linkage words, which are both in colloc_index and no_colloc_index. 
        
        '''
        
        if len(df)==None:
            df= self.df
            
        
        col_index, no_col_index,_ = self.findIndex(colloc_word,df)
#         col_index, no_col_index,_ = self.findIndexListType(colloc_word,df)
        
    
                                  
        colloc_linkage_word = set(sum(list(strToList(df).loc[col_index,'Linkage']),[]))
        no_colloc_linkage_word = set(sum(list(strToList(df).loc[no_col_index,'Linkage']),[]))    
        
        
        # 추가 수정 - 연어와 그 분절된 단어가 같이 있을 경우 duplicated word에서 제거
        # string으로 추가하기에 이런 오류가 발생하는가?
        
        duplicated_word = [word for word in no_colloc_linkage_word if (word in colloc_linkage_word)]
#         duplicated_word = [word for word in no_colloc_linkage_word if (word in colloc_linkage_word) and (word not in colloc_word.split('_'))]
       
        # 추가 작업 - duplicated 단어 중에 분리 시 colloc_word와 겹치는 부분이 있는지 확인하고 제거
        dup_list=[]
        for word in duplicated_word:
            chk=0
            for chunk in word.split('_'):
                if chunk not in colloc_word.split('_'):
                    chk += 0
                else:
                    chk+=1
                    print("dup_list pass: ",chunk, "'in'", word)
                    
            if chk==0:
                dup_list.append(word)
                    
        
        return dup_list,duplicated_word
    
    
    def SumDrop(self,colloc_word,linkage_word,df_colloc, df_no_colloc):
        '''
        params : linkage_word - A word which among the result of findLinkageWord function (duplicated_word).
        return : sum weight to colloc_word , and delete the row in no_colloc_word
        '''
        
        colloc_row = [index for index, data in enumerate(df_colloc.loc[:,'Linkage']) if linkage_word in data][0]
#         print("colloc_row: ",colloc_row)
        
        no_colloc_row = [index for index, data in enumerate(df_no_colloc.loc[:,'Linkage']) if linkage_word in data][0]
    
#         print(colloc_row,no_colloc_row)
        
        df_colloc.iloc[colloc_row,1]+=df_no_colloc.iloc[no_colloc_row,1].item()
        df_no_colloc = df_no_colloc.drop(df_no_colloc.index[no_colloc_row],axis=0)

        return df_colloc, df_no_colloc

    
    def SumDropAll(self,colloc_word,duplicated_word,df_colloc, df_no_colloc):
        
#         colloc_index, no_colloc_index = self.findIndex(colloc_word)
#         df_no_colloc = self.strToList().iloc[no_colloc_index,:]
#         df_colloc = self.strToList().iloc[colloc_index,:]
        
        
        # 새로운 colloc_word마다 새롭게 형성
        a = df_colloc
        b = df_no_colloc

        for linkage_word in duplicated_word:
#             print(linkage_word)
            a,b = self.SumDrop(colloc_word,linkage_word,a,b)
#             print("df_colloc 길이: ",len(a), "df_no_colloc 길이: ",len(b))
        
        return a, b
    

In [0]:
import ast

def strToList(df):
    '''
    desc : Changing df['linkage'] data type from string to list.

    '''
    try :
        df['Linkage']= [ast.literal_eval(str) for str in df.iloc[:,0]]

    except:
    #             print("Already Transformed to list")
        pass

    return df


## collocation word로 weight 몰아주기

1) 모든 corpus 파일을 MergeColloc class에서 처리

2) 각 corpus 마다 collocWord를 찾음

3) 각 collocWord마다 findIndex를 통해서 몰아주기

In [0]:
import warnings
warnings.filterwarnings('ignore')

import os
os.listdir('cooc/')[:5]

['17418.csv', '17419.csv', '17420.csv', '17421.csv', '17422.csv']

In [0]:
temp = pd.read_csv('cooc/17669.csv',index_col=0)
# temp
# print([index for index, linkage in enumerate(temp.iloc[:,0]) if 'Bill_Weidner' in linkage])

In [0]:
c = MergeColloc('17669.csv')   
print(c.colloc_words)
colloc_index, no_colloc_index,_ = c.findIndex('Anthony_Weiner',df=c.df)

['Fox_News', 'John_Podesta', 'Justice_Department', 'FBI_Memo', 'Anthony_Weiner', 'Oversight_Committee', 'Nick_Ackerman', 'Director_Comey', 'Donald_Trump', 'House_Oversight', 'Matthew_Miller', 'United_States', 'White_House', 'New_York', 'Hillary_Clintons', 'Huma_Abedin', 'Director_James', 'Senior_Editor', 'Gordon_Duff']


In [0]:
print(colloc_index)
print(no_colloc_index)

[3083, 3212, 3211, 3210, 3209, 3208, 3115, 3131, 3066, 3099, 3207, 3188, 3201, 3202, 3203, 3204, 3205, 3206, 3175, 3147, 3161, 3045]
[3093, 3255, 3248, 3261, 3266, 3270, 3273, 3275, 3222, 3231, 3240, 3277, 3109, 3125, 3056, 3077, 3186, 3199, 3142, 3172, 3157]


In [0]:
c.df

In [0]:
temp = c.df.loc[colloc_index,:]
temp

In [0]:
[i for i in [1,2,3,4] if i in [1,2]]

[1, 2]

In [0]:
temp.loc[[i for i in temp.index if i not in [3175,3147,3161,3045]],:]

,Linkage,Weight
3083,"[Clinton, Anthony_Weiner]",2
3212,"[Anthony_Weiner, wife]",1
3211,"[Anthony_Weiner, Weiners]",1
3210,"[Anthony_Weiner, Huma_Abedin]",1
3209,"[Anthony_Weiner, aide]",1
3208,"[Anthony_Weiner, top]",1
3115,"[find, Anthony_Weiner]",1
3131,"[computer, Anthony_Weiner]",1
3066,"[focus, Anthony_Weiner]",1
3099,"[email, Anthony_Weiner]",1


In [0]:
strToList(c.df).drop(df.index[colloc_index+no_colloc_index],axis=0)
        

Int64Index([  48,  455,  543,  552,  224,  541, 1018, 1021, 1177,  217,
            ...
            2171, 2172, 2173, 2174, 2175, 2176, 2177, 2178, 2179, 5352],
           dtype='int64', length=5310)

In [0]:
c.df.loc[no_colloc_index,'Linkage']
# list(c.df.loc[colloc_index]['Linkage'])

3093          ['Clinton', 'Weiners']
3255         ['explicit', 'Weiners']
3248         ['sexually', 'Weiners']
3261          ['message', 'Weiners']
3266            ['minor', 'Weiners']
3270              ['top', 'Weiners']
3273             ['aide', 'Weiners']
3275      ['Huma_Abedin', 'Weiners']
3222           ['D-N.Y.', 'Weiners']
3231    ['investigation', 'Weiners']
3240             ['send', 'Weiners']
3277             ['Weiners', 'wife']
3109            ['email', 'Weiners']
3125             ['find', 'Weiners']
3056          ['inquiry', 'Weiners']
3077            ['focus', 'Weiners']
3186             ['U.S.', 'Weiners']
3199      ['congressman', 'Weiners']
3142         ['computer', 'Weiners']
3172           ['former', 'Weiners']
3157              ['use', 'Weiners']
Name: Linkage, dtype: object

In [0]:
[linkage for linkage in c.df.iloc[:,0] if 'Anthony_Weiner' in linkage]

["['Clinton', 'Anthony_Weiner']",
 "['Anthony_Weiner', 'wife']",
 "['Anthony_Weiner', 'Weiners']",
 "['Anthony_Weiner', 'Huma_Abedin']",
 "['Anthony_Weiner', 'aide']",
 "['Anthony_Weiner', 'top']",
 "['find', 'Anthony_Weiner']",
 "['computer', 'Anthony_Weiner']",
 "['focus', 'Anthony_Weiner']",
 "['email', 'Anthony_Weiner']",
 "['Anthony_Weiner', 'minor']",
 "['congressman', 'Anthony_Weiner']",
 "['Anthony_Weiner', 'D-N.Y.']",
 "['Anthony_Weiner', 'investigation']",
 "['Anthony_Weiner', 'send']",
 "['Anthony_Weiner', 'sexually']",
 "['Anthony_Weiner', 'explicit']",
 "['Anthony_Weiner', 'message']",
 "['U.S.', 'Anthony_Weiner']",
 "['use', 'Anthony_Weiner']",
 "['former', 'Anthony_Weiner']",
 "['inquiry', 'Anthony_Weiner']"]

In [0]:
print("colloc_index 개수: ",len(colloc_index))
print("no_colloc_index 개수: ",len(no_colloc_index))

colloc_index 개수:  22
no_colloc_index 개수:  21


In [0]:
#두번 실시하게 되면, 연어를 잡지 못함. 원인 파악 필요

duplicated_word,i = c.findLinkageWord('Anthony_Weiner',df=c.df)
# duplicated_word.pop('Americans') 
# duplicated_word = duplicated_word[1:]
# duplicated_word.remove('Saudi-Israel')

print(duplicated_word)
print(i)

['D-N.Y.', 'U.S.', 'top', 'wife', 'send', 'message', 'find', 'focus', 'congressman', 'aide', 'minor', 'Huma_Abedin', 'former', 'Clinton', 'Weiners', 'sexually', 'explicit', 'investigation', 'use', 'inquiry', 'computer', 'email']
['D-N.Y.', 'U.S.', 'top', 'wife', 'send', 'message', 'find', 'focus', 'congressman', 'aide', 'minor', 'Huma_Abedin', 'former', 'Clinton', 'Weiners', 'sexually', 'explicit', 'investigation', 'use', 'inquiry', 'computer', 'email']


연어와 그 분절된 단어가 하나의 linkage에 같이 있을 경우 오류가 생김,,,

In [0]:
a = c.strToList().iloc[colloc_index,:]
b = c.strToList().iloc[no_colloc_index,:]
d = c.strToList().drop(c.strToList().index[colloc_index+no_colloc_index],axis=0)

colloc_df, no_colloc_df = c.SumDropAll('Anthony_Weiner',duplicated_word,a,b)

In [0]:
colloc_df, no_colloc_df =c.SumDrop('Anthony_Weiner','Weiners',a,b)

In [0]:
no_colloc_df

## collocation에 없는 linkage들 처리

In [0]:
no_colloc_df.iloc[:,0] =pd.Series([ast.literal_eval(str(linkage).replace('Trump','Donald_Trump')) for linkage in no_colloc_df.iloc[:,0]],index=no_colloc_df.index)
no_colloc_df.iloc[:,0] =pd.Series([ast.literal_eval(str(linkage).replace('Donald_Trumps','Donald_Trump')) for linkage in no_colloc_df.iloc[:,0]],index=no_colloc_df.index)

In [0]:
print(set(sum(list(no_colloc_df.iloc[:,0]),[])))
data= pd.concat([colloc_df,no_colloc_df,d])

set()


In [0]:
len(data) #기존 data 1221 , SumDrop을 통해서 5개는 기존의 것에 통합됨

6180

## 전체 파일 대상 변환 후 저장

한계 

- 두 가지로만 된 연어만 처리
- 연어를 replace하는 것에 일반화된 규칙 적용 어려움
- 오류가 뜨는 데이터만 따로 다뤄주는 것이 좋음

=> 주로 뜨는 오류 : duplicated_word에 연어의 부분이 들어가서 문제가 생김. 

=> manual하게 duplicated_word를 수정해서 해결


In [0]:
import os
os.listdir('cooc/')[527:550]

['17945.csv',
 '17946.csv',
 '17947.csv',
 '17948.csv',
 '17949.csv',
 '17950.csv',
 '17951.csv',
 '17952.csv',
 '17953.csv',
 '17954.csv',
 '17955.csv',
 '17956.csv',
 '17957.csv',
 '17958.csv',
 '17959.csv',
 '17960.csv',
 '17961.csv',
 '17962.csv',
 '17963.csv',
 '17964.csv',
 '17965.csv',
 '17966.csv',
 '17967.csv']

In [0]:
# for total files
from tqdm import tqdm_notebook
import os
import time
import warnings
warnings.filterwarnings('ignore')

list_dir = os.listdir('cooc/')[527:]

for data in tqdm_notebook(list_dir,desc='csv file'):
#     data='17669.csv'
    start = time.time()
    print(data)
    first_class = MergeColloc(data)
    
    df = first_class.df
    print("start DF length !: ",len(df))
    
    print("colloc_words list: ",first_class.colloc_words)
    for index, colloc_word in tqdm_notebook(enumerate(first_class.colloc_words),desc ='colloc_words'):
        # 추가수정- duplicated word 제대로 형성 위해서 다시 선언
        c = MergeColloc(data)
#         print("Start df: ",len(df))
#         print("colloc_word: ",colloc_word)
        
        # findindex 자체를 바꿀 가능성 생각해보기
        colloc_index, no_colloc_index,_ = c.findIndex(colloc_word,df=df)
#         print("colloc_index 개수: ",len(colloc_index),"no_collec_index 개수: ",len(no_colloc_index))
        
        duplicated_word,second = c.findLinkageWord(colloc_word,df=df)
#         print("duplicated_word: ",duplicated_word)
#         print("dup_second: ",second)
        
        # 추가수정
#         if colloc_word =='Saudi_Arabias':
#             duplicated_word.remove('Saudi-Israel')
        
        # 여러 colloc_word에 대해서 변경을 계속 축적하려면 row 단위가 아닌 index 단위 변경이 필요함
        a = strToList(df).loc[colloc_index,:]
        b = strToList(df).loc[no_colloc_index,:]
        
        # colloc, no_colloc index가 둘다 없는 index 들
#         d = strToList(df).drop(df.index[colloc_index+no_colloc_index],axis=0)
        
        d = strToList(df).loc[[index for index in df.index if index not in colloc_index + no_colloc_index],:]
            
            
        colloc_df, no_colloc_df = c.SumDropAll(colloc_word,duplicated_word,a,b)
        
        
        #연어 처리하는 일반화된 규칙을 고민해봐야함
        no_colloc_df.iloc[:,0] =pd.Series([ast.literal_eval(str(linkage).replace(colloc_word.split('_')[0],colloc_word)) for linkage in no_colloc_df.iloc[:,0]],index=no_colloc_df.index)
        
    
        df= pd.concat([colloc_df,no_colloc_df,d])
#         print("After for문: ",len(df))
#         print("--------------------------------------------------------------")
#         print(df.head(10))
    
    print("End DF length !: ",len(df))
    os.system('mkdir re_cooc/')
    df.to_csv('re_cooc/' + data)
    end = time.time()
    print("It takes :", end - start)
    print("---------------------------------------------------")
#     break
    
    
    
        

17945.csv
start DF length !:  6571
colloc_words list:  ['Autonomous_Republic', 'Godless_Communists', 'US_Army', 'Krimean_Peninsula', 'Soviet_Army', 'New_York', 'Mediterranean_Sea', 'United_States', 'Soviet_Union', 'Godless_Russians', 'Mr._Trump', 'Mr._Trumps', 'US_Navy', 'US_Military', 'Ukraine_SSR', 'John_Teft', 'Auslander_Author', 'Border_Control', 'Pax_Rossiya', 'Geoffrey_Pyatt', 'Politically_Correct', 'White_House']


End DF length !:  6560
It takes : 93.81780481338501
---------------------------------------------------
17946.csv
start DF length !:  4583
colloc_words list:  ['Daily_Beast', 'Thurgood_Marshall', 'Joseph_Lowery', 'Langston_Hughes', 'Barack_Obama', 'Washington_Post', 'Northern_District', 'James_Baldwin', 'Remove_Black', 'Malala_Yousafzai', 'Andrea_Burton', 'Rosa_Parks', 'Civil_Rights', 'Equal_Protection', 'Blacks_Law', 'Robert_Milich', 'Due_Process', 'Akron_School', 'Tweet_Widget', 'Justice_Justice', 'United_States', 'Sojourner_Truth', 'Bill_Quigley', 'Federalist_Papers', 'Ralph_Abernathy', 'Alexander_Hamilton', 'Georgetown_University', 'Youngstown_Ohio']


dup_list pass:  Obama 'in' Obama
End DF length !:  4554
It takes : 82.39840078353882
---------------------------------------------------
17947.csv
start DF length !:  4583
colloc_words list:  ['Daily_Beast', 'Thurgood_Marshall', 'Joseph_Lowery', 'Langston_Hughes', 'Barack_Obama', 'Washington_Post', 'Northern_District', 'James_Baldwin', 'Remove_Black', 'Malala_Yousafzai', 'Andrea_Burton', 'Rosa_Parks', 'Civil_Rights', 'Equal_Protection', 'Blacks_Law', 'Robert_Milich', 'Due_Process', 'Akron_School', 'Tweet_Widget', 'Justice_Justice', 'United_States', 'Sojourner_Truth', 'Bill_Quigley', 'Federalist_Papers', 'Ralph_Abernathy', 'Alexander_Hamilton', 'Georgetown_University', 'Youngstown_Ohio']


dup_list pass:  Obama 'in' Obama
End DF length !:  4554
It takes : 82.92501616477966
---------------------------------------------------
17948.csv
start DF length !:  5814
colloc_words list:  ['Polls_Fraudulent', 'Ric_Militi', 'Millenium_Report', 'Gateway_Pundit', 'Hillary_Clinton', 'Adolf_Hitler', 'Comrade_Clinton', 'Bernie_Sanders', 'Mein_Kampf', 'Crazy_Raccoons', 'Donald_Trump', 'Breitbart_News', 'Anonymous_Patriots', 'Camp_Clinton']


dup_list pass:  Anonymous 'in' Anonymous
End DF length !:  5789
It takes : 53.77348566055298
---------------------------------------------------
17949.csv
start DF length !:  5923
colloc_words list:  ['House_Nancy', 'Global_Climate', 'Emma_Thompson', 'Climate_Change', 'Democratic_Speaker', 'Republican_Speaker', 'Real_News', 'Will_Rise', 'Larry_Wilkerson', 'George_Monbiot', 'Climate_Science', 'Rowan_Wolf', 'Climate_Bureau', 'Media_Matters', 'Greanville_Post', 'United_States', 'Intergovernmental_Panel', 'Koch_Brothers', 'Michael_Mann', 'Dahr_Jamail', 'Paris_Agreement']


...

dup_list pass:  Ted 'in' Ted
dup_list pass:  Turner 'in' Turner
dup_list pass:  Treasury 'in' Treasury
End DF length !:  9505
It takes : 239.21100854873657
---------------------------------------------------
18171.csv
start DF length !:  9876
colloc_words list:  ['Citizens_United', 'Jill_Stein', 'Vanity_Fair', 'Seth_Meyers', 'Ralph_Nader', 'Ronald_Reagan', 'Fidel_Castro', 'Washington_Post', 'Bernie_Bros', 'Edward_Snowden', 'Bernie_Sanders', 'Does_Eichenwald', 'United_States', 'South_Carolina', 'Donald_Trump', 'Kurt_Eichenwald', 'Carl_Bialik', 'Nate_Cohn', 'Hong_Kong', 'Green_Party']


End DF length !:  9849
It takes : 129.17053651809692
---------------------------------------------------
18172.csv
start DF length !:  8094
colloc_words list:  ['Human_Derivatives', 'Stewart_Dougherty', 'Deep_States', 'Deep_State', 'Legal_Tender', 'Apple_Computer', 'Inferential_Analytics', 'Financial_Markets', 'United_States', 'Market_Manipulation', 'Larry_Summers', 'Oligarchs_Plan', 'Craigs_List', 'Freedom_Fighters', 'Tufts_University', 'Narendra_Modi', 'Apple_Pay']


End DF length !:  8075
It takes : 88.87670111656189
---------------------------------------------------
18173.csv
start DF length !:  7177
colloc_words list:  ['Child_Below', 'Jen_Hello', 'Children_If', 'NutritionistMia_Agreed', 'DashnMe_Ok', 'Julie_Same', 'Bobbi_Sorry', 'Womens_Hospital', 'Keri_Hanson', 'Maria_Oh', 'Harvards_Brigham', 'Julie_Yeah', 'Foroogh_NF', 'Julie_So']


dup_list pass:  Child 'in' Child
dup_list pass:  Maria 'in' Maria
End DF length !:  7116
It takes : 65.26454401016235
---------------------------------------------------
18174.csv
start DF length !:  87749
colloc_words list:  ['Land_Warfare', 'Charlemagne_Pralte', 'Field_Manual', 'Mohammad_Mosaddegh', 'Homeland_Security', 'Say_Something', 'War_Department', 'Donald_Trump', 'Rahul_Manchanda', 'HMAS_Sydney', 'Regime_Change', 'George_Bush', 'Second_Gulf', 'Korean_War', 'Tonkin_Incident', 'Mau_Mau', 'United_States', 'Maurice_Bishop', 'American_People', 'US_Department', 'Panama_Canal', 'Red_Crescent', 'CIA_Asset', 'Saddam_Hussein', 'Populist_Revolt', 'Reinhard_Heydrich', 'North_Korea', 'Hillary_Clinton', 'RMS_Carmania', 'Federal_Bureau', 'Adolf_Hitler', 'Pearl_Harbor', 'Otto_Skorzeny', 'South_Korea', 'Dachau_Trials', 'Geneva_Conventions', 'Communist_Party', 'Clinton_Administration', 'See_Something', 'Hearst_Press', 'Red_Cross', 'Fidel_Castros', 'Middle_East', 'United_Nations']


dup_list pass:  Bush 'in' Bush
dup_list pass:  Gulf 'in' Gulf
dup_list pass:  Korean 'in' Korean
dup_list pass:  Tonkin 'in' Tonkin
dup_list pass:  Mau 'in' Mau
dup_list pass:  People 'in' People
dup_list pass:  Department 'in' War_Department
dup_list pass:  Department 'in' Department
dup_list pass:  Red 'in' Red_Cross
dup_list pass:  Red 'in' Red
dup_list pass:  Saddam 'in' Saddam
dup_list pass:  North 'in' North
dup_list pass:  Korea 'in' South_Korea
dup_list pass:  Carmania 'in' Carmania
dup_list pass:  Skorzeny 'in' Skorzeny
dup_list pass:  Communist 'in' Communist
dup_list pass:  Red 'in' Red_Crescent


## New Function


> colloc_index: Target 연어가 들어가 있는 index

=> no_colloc_index에서 linkage를 합할 것이 있다면 weight를 추가

> no_colloc_index : Target 연어가 분리 된 것이 하나라도 들어있는 index

1) colloc_index에도 있는 단어가 있을 경우 colloc_index의 weight에 가지고 있던 weight를 합하고 삭제

2) colloc_index에는 없는 단어가 있을 경우
Target 연어가 합쳐져 있는 것으로 대상을 바꿔줌


In [0]:
# c.strToList().iloc[colloc_index,0]
colloc_linkage_word = set(sum(list(c.strToList().iloc[colloc_index,0]),[]))-set(['Donald_Trump'])
no_colloc_linkage_word = set(sum(list(c.strToList().iloc[no_colloc_index,0]),[]))

print("colloc_linkage_word")
print(colloc_linkage_word)
print("no_colloc_linkage_word")
print(no_colloc_linkage_word)

colloc_linkage_word
{'candidate', 'largely', 'female', 'Sunday', 'assertion', 'base', 'success', 'campaign', 'criticism', 'Hillary_Clintons', 'soft', 'foreign', 'dismissed', 'presidential', 'Republican', 'defend', 'policy'}
no_colloc_linkage_word
{'also', 'strategy', 'qualification', 'need', 'female', 'face', 'approach', 'focus', 'spending', 'continue', 'woman', 'attack', 'low', 'number', 'speech', 'weak', 'Vermont', 'former', 'knock', 'lead', 'criticism', 'Middle_East', 'tell', 'challenger', 'primary', 'instability', 'Fox_News', 'secretary', 'policy', 'failed', 'naive', 'Texas', 'scale', 'question', 'Democrat', 'dictator', 'However', 'get', 'Sunday', 'suggest', 'say', 'rival', 'sound', 'front-running', 'Week', 'Cruz', 'week', 'Trumps', 'Bernie', 'liberal', 'nevertheless', 'voter', 'argue', 'election', 'domestic', 'Trump', 'overthrow', 'foreign', 'especially', 'Ted', 'Clinton', 'bad', 'Sen.', 'general', 'favorability', 'GOP', 'card', 'improve', 'state', 'reflect', 'back', 'last', 'ISIS

In [0]:
# [word for word in colloc_linkage_word if word in no_colloc_linkage_word]

[word for word in no_colloc_linkage_word if word in colloc_linkage_word]

['female', 'criticism', 'policy', 'Sunday', 'foreign']

In [0]:
df_no_colloc = c.strToList().iloc[no_colloc_index,:]
df_colloc = c.strToList().iloc[colloc_index,:]

linkage_word = 'Sunday'
df_colloc.iloc[[index for index, data in enumerate(df_colloc.loc[:,'Linkage']) if linkage_word in data],:]

,Linkage,Weight
48,"[Donald_Trump, Sunday]",1


In [0]:
print([index for index, data in enumerate(df_colloc.loc[:,'Linkage']) if linkage_word in data])
print([index for index, data in enumerate(df_no_colloc.loc[:,'Linkage']) if linkage_word in data])

[16]
[5]


In [0]:
df_no_colloc.iloc[[index for index, data in enumerate(df_no_colloc.loc[:,'Linkage']) if linkage_word in data],:]

In [0]:
df_no_colloc.iloc[5,:]

In [0]:
colloc_index = 48
no_colloc_index = 164

df_colloc.loc[df_colloc.index==colloc_index,'Weight']+=df_no_colloc.loc[df_no_colloc.index ==no_colloc_index,'Weight'].item()
df_no_colloc = df_no_colloc.drop(index=no_colloc_index)

## 이후 코드 진행 필요

- Features.py 수정본 적용하기
- df 에 새로운 열 feature 들 추가하기 

### Features.py (networkx 피쳐 일부 추가)

 * 현재 density, degree mean,var 추가
 * degree는 만들긴 했는데 기사마다 너무 달라서 안 쓰는게 더 좋을지도
  
 
 저 없을때 ti-cnn csv에서 말고 따로 피쳐 구현한게 있다면 추가해주세요

In [0]:
"""
Meta information(=features for machine learning) extraction

June 2nd, 2019
author: Jin Uk, Cho
"""

import sklearn
import networkx as nx
import pandas as pd
import numpy as np

from network_visualization import Graph
from reweight import Reweight


# 척도 계산하기
class Measure():
    def __init__(self, graph):
        self.graph = graph

    def cal_Cent(self, g):
        deg_cent = nx.algorithms.degree_centrality(g)
        clo_cent = nx.algorithms.closeness_centrality(g)
        bet_cent = nx.algorithms.betweenness_centrality(g)
        eig_cent = nx.algorithms.eigenvector_centrality_numpy(g)
        # info_cent = nx.algorithms.information_centrality(g)
        list_deg = [k for k in deg_cent.values()]
        self.list_deg = list_deg
        list_clo = [k for k in clo_cent.values()]
        self.list_clo = list_clo
        list_bet = [k for k in bet_cent.values()]
        self.list_bet = list_bet
        list_eig = [k for k in eig_cent.values()]
        self.list_eig = list_eig

        # return deg_cent

    def get_Info(self):
        summary_g = nx.info(self.graph)
        print(summary_g)
        return summary_g

    def create_Dataframe(self):
        """
        :return: (dataframe) table of info of each nodes
        """
        word = pd.Series(self.deg_cent.keys())
        deg_val = pd.Series(self.deg_cent.values())
        bet_val = pd.Series(self.bet_cent.values())
        clo_val = pd.Series(self.clo_cent.values())
        eig_val = pd.Series(self.eig_cent.values())
        dataframe = pd.DataFrame({'Word': word, 'Betweeness': bet_val, 'Degree': deg_val,
                                  'Closeness': clo_val,'Eigen':eig_val})
        return dataframe

    def deg_GroupVal(self):
        """
        :param (list) list of values:
        Cd = Summation(Max - i th value) / (g-2)(g-1)
        :return: (float)  grouped value
        """
        X = 0
        max_val = max(self.list_deg)
        g = len(self.list_deg)
        gg = (g - 2) * (g - 1)
        print('maximum value of degree cetrality is : {0}'.format(max_val))
        for i in self.list_deg:
            tmp = max_val - i
            X += tmp
        result = X / gg
        print(result)
        return result

    def clo_GroupVal(self):
        """
        :param (list) list of values:
        Cd = Summation(Max - i th value) / ((g-2)(g-1)/(2g-3))
        :return: (float)  grouped value
        """
        X = 0
        max_val = max(self.list_clo)
        g = len(self.list_clo)
        gg = ((g - 2) * (g - 1)) / (2 * g - 3)
        print('maximum value of closeness cetrality is : {0}'.format(max_val))
        for i in self.list_clo:
            tmp = max_val - i
            X += tmp
        result = X / gg
        print(result)
        return result

    def bet_GroupVal(self):  ###########
        """
        :param (list) list of values:
        Cd = Summation(Max - i th value) / ((g-2)^2(g-1)/2)
        :return: (float)  grouped value
        """
        X = 0
        max_val = max(self.list_bet)
        g = len(self.list_bet)
        gg = (pow((g - 2), 2) * (g - 1)) / (2)
        # print('maximum value of betweenness cetrality is : {0}'.format(max_val))
        for i in self.list_bet:
            tmp = max_val - i
            X += tmp
        result = X / gg
        # print(result)
        return result
    
    def eig_GroupVal(self):  ###########
        """
        :param (list) list of values:
        Cd = Summation(Max - i th value) / ((g-2)^2(g-1)/2)
        :return: (float)  grouped value
        """
        X = 0
        max_val = max(self.list_eig)
        g = len(self.list_eig)
        gg = (pow((g - 2), 2) * (g - 1)) / (2)
        # print('maximum value of betweenness cetrality is : {0}'.format(max_val))
        for i in self.list_eig:
            tmp = max_val - i
            X += tmp
        result = X / gg
        # print(result)
        return result


    #
    def get_Value(self):
        """
        calculate all values and return it as a list
        :return: (float) values
        """
        # info = self.get_Info()
        start = self.cal_Cent(self.graph)
        deg_val = self.deg_GroupVal()
        clo_val = self.clo_GroupVal()
        bet_val = self.bet_GroupVal()
        eig_val = self.eig_GroupVal()

        return deg_val,clo_val,bet_val,eig_val


class Feature():
    def __init__(self, doc_path_list):
        self.doc_filenames = doc_path_list
        # self.df_tfidf = pd.read_csv(doc_path_list[0][:-20] + 'tfidf.csv', index_col=0)
#         self.df_tfidf = pd.read_csv('data/tfidf.csv', index_col=0)

#     def cal_tfidf(self):
#         tfidf_mean = np.mean(self.df_tfidf['Tfidf'])
#         tfidf_var = np.var(self.df_tfidf['Tfidf'])
#         return tfidf_mean, tfidf_var

    def cal_edge_weight(self, matrix):
        wt_mean = np.mean(matrix['Weight'])
        wt_var = np.var(matrix['Weight'])
        return wt_mean, wt_var

    def cal_edge_num(self, matrix):
        return len(matrix['Linkage'])

    def cal_net_feature(self, G):
        '''
        > mean, variance 
        betweeness / degree / common_neighbors /
        
        > core_count / dens / 
        '''
        net = Measure(G)
        bet_val = net.get_Value()
        
        deg_hist = nx.degree_histogram(G)
        deg_mean = 0
        for i in range(len(deg_hist)):
            deg_mean = deg_mean + i*deg_hist[i]
        deg_mean = deg_mean / np.sum(np.array(deg_hist))
        deg_var = 0
        for i in range(len(deg_hist)):
            deg_var = deg_var + ((i-deg_mean)**2)*deg_hist[i]
        deg_var = deg_var / np.sum(np.array(deg_hist))
        
        common_neighbors = [len(list(nx.common_neighbors(G, u, v))) for u, v in G.edges]
        com_mean = np.mean(np.array(common_neighbors))
        com_var = np.var(np.array(common_neighbors))
        
        #
        deg_hist = nx.degree_histogram(G)
        eig_mean = 0
        for i in range(len(deg_hist)):
            deg_mean = deg_mean + i*deg_hist[i]
        deg_mean = deg_mean / np.sum(np.array(deg_hist))
        deg_var = 0
        for i in range(len(deg_hist)):
            deg_var = deg_var + ((i-deg_mean)**2)*deg_hist[i]
        deg_var = deg_var / np.sum(np.array(deg_hist))
        #
        
        
        degree_sequence = sorted([d for n, d in G.degree()], reverse=True)
        core_count = len([i for i in degree_sequence if i > np.quantile(degree_sequence, 0.75)])
        dens = nx.density(G)
        
        
        return deg_mean, deg_var, com_mean, com_var, core_count, bet_val, dens,

    def make_df(self, doc_path, label):
        net = Graph()
        G, matrix = net.create_graph(doc_path, string_to_list=True)  # 이미 tfidf_reweight.csv 로 된 애들을 만들어놔서 그걸로 시작해야함
        tfidf_mean, tfidf_var = self.cal_tfidf()
        wt_mean, wt_var = self.cal_edge_weight(matrix)
        edge_num = self.cal_edge_num(matrix)
        deg_mean, deg_var, com_mean, com_var, core_count, bet_val, dens = self.cal_net_feature(G)

        feature_df_one = {'tfidf_mean': tfidf_mean,
                          'tfidf_var': tfidf_var,
                          'wt_mean': wt_mean,
                          'wt_var': wt_var,
                          'edge_num': edge_num,
                          'degree_mean': deg_mean,
                          'degree_var': deg_var,
                          'com_mean': com_mean,
                          'com_var': com_var,
                          'core_count': core_count,
                          'betweeness': bet_val,
                          'density' : dens,
                          'label': label, 'index': doc_path[-20:-4]}

        return feature_df_one

    def make_df_from_dataset(self):
        idx_list = []
        row_list = []
#        for doc_path in self.doc_filenames[:58]:
#            idx_list.append(doc_path[-20:-4]) # title of article
#            row_list.append(self.make_df(doc_path, label=0))
        for doc_path in self.doc_filenames[:5]:
            print(doc_path[-20:-4])
            idx_list.append(doc_path[-20:-4])  # title of article
            row_list.append(self.make_df(doc_path, label=1))

        feature_df = pd.DataFrame(row_list, columns=row_list[0].keys(), index=idx_list)

        return feature_df